In [23]:
import os
import cv2 as cv
import numpy as np
import pickle
from PIL import Image

In [24]:
#declare the directories
base_dir = os.path.dirname(os.path.abspath("__file__"))
#print(base_dir)
image_dir = os.path.join(base_dir, "images")
# image_dir= '/Users/scottliu/Desktop/train'

In [25]:
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml') #Face Classifier
recognizer = cv.face.LBPHFaceRecognizer_create()# if the recognizer gives you errors then
#run this: pip uninstall opencv-contrib-python
#then run this: pip install --user opencv-contrib-python    (you could remove the --user)
current_id = 0
label_ids = {} #diccionary containing all the people I have identified
y_labels = [] #list of numbers related to the labels
x_train = [] #picture values

# Troubleshooting

Very often the cell above will not give you trouble, in this case you can ignore this message. However, if this line: 

```recognizer = cv.face.LBPHFaceRecognizer_create()```

gives you an error, you may have conflicting libraries. In which case, you should uninstall the opencv-contrib-python by using this code: 

```pip uninstall opencv-contrib-python``` 

from the Python terminal. Then, reinstall the following line: 

```pip install --user opencv-contrib-python    (you could remove the --user)```

In [26]:
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("jpg") or file.endswith("png"):
            path = os.path.join(root, file)
            label = os.path.basename(path).replace(".jpg", "").replace(".png", "").replace(" ", "-").lower()
            print (label, path)
            
            if label not in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_now = label_ids[label]
            print(label_ids)
                           
            pil_image = Image.open(path).convert("L") #open image and convert it to grayscale.
            image_array = np.array(pil_image, "uint8")
            print(image_array)
            
            # Detect faces in the image
            faces = face_cascade.detectMultiScale(
                image_array,
                scaleFactor=1.21, #1.25
                minNeighbors=4, #5
                minSize=(20, 20), #30, 30
                flags = cv.CASCADE_SCALE_IMAGE
            )
            
            # Draw a rectangle around the faces
            for (x, y, w, h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi) #this is our training data
                y_labels.append(id_now)

# save all labels as a "labels.pickle" file (in the same directory)
with open("labels.pickle", 'wb') as file: #'wb' stands for "writing bytes"
    pickle.dump(label_ids, file)

recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")
print("DONE!")

4 /Users/scottliu/Desktop/W13 Image Recognition/images/4.jpg
{'4': 0}
[[220 220 220 ... 160 159 159]
 [219 219 220 ... 160 159 159]
 [220 220 220 ... 160 160 160]
 ...
 [ 25  25  25 ...  21  21  21]
 [ 23  23  23 ...  20  21  21]
 [ 21  21  22 ...  19  21  21]]
5 /Users/scottliu/Desktop/W13 Image Recognition/images/5.jpg
{'4': 0, '5': 1}
[[225 225 225 ... 163 164 163]
 [225 225 225 ... 163 164 163]
 [225 225 225 ... 163 164 163]
 ...
 [ 25  25  25 ...  22  22  22]
 [ 25  25  25 ...  22  21  20]
 [ 25  25  25 ...  22  22  19]]
2 /Users/scottliu/Desktop/W13 Image Recognition/images/2.jpg
{'4': 0, '5': 1, '2': 2}
[[225 225 225 ... 163 163 163]
 [225 225 225 ... 163 163 163]
 [225 225 225 ... 163 163 163]
 ...
 [ 27  27  27 ... 178 178 177]
 [ 25  25  25 ... 168 174 177]
 [ 25  25  25 ... 144 162 173]]
3 /Users/scottliu/Desktop/W13 Image Recognition/images/3.jpg
{'4': 0, '5': 1, '2': 2, '3': 3}
[[220 220 220 ... 155 155 155]
 [220 220 220 ... 155 155 155]
 [220 220 219 ... 155 155 155]
 ..